In [37]:
# import findspark
# findspark.init()

# import pyspark
# import random

# sc = pyspark.SparkContext(appName="Pi")
# num_samples = 100000000

# def inside(p):     
#   x, y = random.random(), random.random()
#   return x*x + y*y < 1

# count = sc.parallelize(range(0, num_samples)).filter(inside).count()

# pi = 4 * count / num_samples
# print(pi)

# sc.stop()

# To run spark on jupyter

In [1]:
import findspark
findspark.init()

import pyspark
import random

# Load events

In [49]:
import json
with open('event_10_2018.json') as json_file:
    data = json.load(json_file)
#     data = data['data']
print(data[0])

{'code': '060', 'src_actor': 'RUS', 'month': '10', 'tgt_actor': '', 'country_code': 'UKR', 'year': '2018', 'date8_val': {'$date': 1539302400000}, 'id': '5bc0e3beff261b4fdacb191b_3', 'source': 'RUS', 'date8': '20181012', 'src_agent': '', 'latitude': 49.0, 'src_other_agent': '', 'geoname': 'Ukraine ', 'quad_class': 2, 'source_text': 'bbc', 'root_code': '06', 'tgt_other_agent': '', 'day': '12', 'target': '---MIL', 'goldstein': 6.0, 'tgt_agent': 'MIL', 'longitude': 32.0, 'url': 'https://www.bbc.co.uk/news/world-europe-45834197', '_id': {'$oid': '5bc0e4c7583dca39fd0d583d'}}


In [35]:
# checking data
data[0]

{'code': '033',
 'src_actor': 'USA',
 'month': '10',
 'tgt_actor': 'USA',
 'country_code': 'USA',
 'year': '2017',
 'date8_val': {'$date': 1509408000000},
 'id': '59f7feb4de7923401d8a5d7a_1',
 'source': 'USA',
 'date8': '20171031',
 'src_agent': '',
 'latitude': 21.30694,
 'src_other_agent': '',
 'geoname': 'Honolulu Hawaii',
 'quad_class': 1,
 'source_text': 'wn_world',
 'root_code': '03',
 'tgt_other_agent': '',
 'day': '31',
 'target': 'USAGOV',
 'goldstein': 5.2,
 'tgt_agent': 'GOV',
 'longitude': -157.85833,
 'url': 'https://article.wn.com/view/2017/10/31/6_states_seek_to_intervene_in_Hawaiis_travel_ban_challenge_e2/',
 '_id': {'$oid': '59f7fffe583dca26c729493a'}}

# Write it to an out file for proper reading

In [51]:

for i in range(len(data)):
    with open("out.json",'a') as file:
        file.write(json.dumps(data[i]))
        file.write("\n")  
    file.close()


# Load libraries

In [52]:
from math import sqrt
import numpy as np
from numpy import array
import pyspark
from pyspark import *
from pyspark.conf import *
from pyspark.sql import *
from pyspark import SparkContext
from pyspark import SparkConf, SparkContext
import pyspark.sql 
from pyspark.sql import SQLContext
from pyspark.sql.types import *

# sc = spark.sparkContext

In [344]:
conf = SparkConf().setAppName("News").setMaster("local")
spark = SparkContext(conf = conf)
session = SparkSession(spark)

# read Json

In [105]:
# df = spark.read.json("out.json")
eventsDF = session.read.json("out.json",multiLine = False)

In [375]:
eventsDF.printSchema()

root
 |-- code: string (nullable = true)
 |-- country_code: string (nullable = true)
 |-- date8: string (nullable = true)
 |-- day: string (nullable = true)
 |-- geoname: string (nullable = true)
 |-- goldstein: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- month: string (nullable = true)
 |-- quad_class: long (nullable = true)
 |-- root_code: string (nullable = true)
 |-- source: string (nullable = true)
 |-- src_actor: string (nullable = true)
 |-- tgt_actor: string (nullable = true)
 |-- year: string (nullable = true)



# Drop columns which might not be useful


In [106]:
eventsDF = eventsDF.drop('date8_val')
eventsDF = eventsDF.drop('id')
eventsDF = eventsDF.drop('_id')
eventsDF = eventsDF.drop('url')
eventsDF = eventsDF.drop('_id')
eventsDF = eventsDF.drop('src_agent')
eventsDF = eventsDF.drop('src_other_agent')
eventsDF = eventsDF.drop('target')
eventsDF = eventsDF.drop('tgt_agent')
eventsDF = eventsDF.drop('tgt_other_agent')
eventsDF = eventsDF.drop('source_text')

In [107]:
eventsDF.show()

+----+------------+--------+----+--------------------+---------+--------+----------+-----+----------+---------+------+---------+---------+----+
|code|country_code|   date8| day|             geoname|goldstein|latitude| longitude|month|quad_class|root_code|source|src_actor|tgt_actor|year|
+----+------------+--------+----+--------------------+---------+--------+----------+-----+----------+---------+------+---------+---------+----+
| 060|         UKR|20181012|  12|            Ukraine |      6.0|    49.0|      32.0|   10|         2|       06|   RUS|      RUS|         |2018|
| 030|         RUS|20181012|  12| Russian Federation |      4.0|    60.0|     100.0|   10|         1|       03|RUSGOV|      RUS|         |2018|
| 010|         CHN|20181012|  12| Beijing Beijing Shi|      0.0| 39.9075| 116.39723|   10|         0|       01|---GOV|         |      CHN|2018|
| 030|         CHN|20181012|  12|            Taishan |      4.0|22.25135|  112.7799|   10|         1|       03|   USA|      USA|      US

In [108]:
eventsDF.count()


27231

# Remove NA values

In [109]:
# import pyspark.sql.*
# DataFrameNaFunctions.drop()
# eventsDF.dropna()
eventsDF = eventsDF.na.drop()

In [110]:
eventsDF.count()

19920

In [104]:
eventsDF.show(1)

+----+------------+--------+---+--------+---------+--------+---------+-----+----------+---------+------+-----------+---------+---------+----+
|code|country_code|   date8|day| geoname|goldstein|latitude|longitude|month|quad_class|root_code|source|source_text|src_actor|tgt_actor|year|
+----+------------+--------+---+--------+---------+--------+---------+-----+----------+---------+------+-----------+---------+---------+----+
| 060|         UKR|20181012| 12|Ukraine |      6.0|    49.0|     32.0|   10|         2|       06|   RUS|        bbc|      RUS|         |2018|
+----+------------+--------+---+--------+---------+--------+---------+-----+----------+---------+------+-----------+---------+---------+----+
only showing top 1 row



# Encode strings

In [126]:
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol="country_code", outputCol="country_idx")
indexed = indexer.fit(eventsDF).transform(eventsDF)
indexed.show()

+----+------------+--------+---+--------------------+---------+--------+----------+-----+----------+---------+------+---------+---------+----+-----------+
|code|country_code|   date8|day|             geoname|goldstein|latitude| longitude|month|quad_class|root_code|source|src_actor|tgt_actor|year|country_idx|
+----+------------+--------+---+--------------------+---------+--------+----------+-----+----------+---------+------+---------+---------+----+-----------+
| 060|         UKR|20181012| 12|            Ukraine |      6.0|    49.0|      32.0|   10|         2|       06|   RUS|      RUS|         |2018|       44.0|
| 030|         RUS|20181012| 12| Russian Federation |      4.0|    60.0|     100.0|   10|         1|       03|RUSGOV|      RUS|         |2018|        9.0|
| 010|         CHN|20181012| 12| Beijing Beijing Shi|      0.0| 39.9075| 116.39723|   10|         0|       01|---GOV|         |      CHN|2018|        6.0|
| 030|         CHN|20181012| 12|            Taishan |      4.0|22.2513

In [127]:
from pyspark.ml.feature import OneHotEncoderEstimator

In [128]:
indexer = StringIndexer(inputCol="geoname", outputCol="gname")
indexed = indexer.fit(indexed).transform(indexed)
indexed.show()

+----+------------+--------+---+--------------------+---------+--------+----------+-----+----------+---------+------+---------+---------+----+-----------+------+
|code|country_code|   date8|day|             geoname|goldstein|latitude| longitude|month|quad_class|root_code|source|src_actor|tgt_actor|year|country_idx| gname|
+----+------------+--------+---+--------------------+---------+--------+----------+-----+----------+---------+------+---------+---------+----+-----------+------+
| 060|         UKR|20181012| 12|            Ukraine |      6.0|    49.0|      32.0|   10|         2|       06|   RUS|      RUS|         |2018|       44.0| 129.0|
| 030|         RUS|20181012| 12| Russian Federation |      4.0|    60.0|     100.0|   10|         1|       03|RUSGOV|      RUS|         |2018|        9.0|  22.0|
| 010|         CHN|20181012| 12| Beijing Beijing Shi|      0.0| 39.9075| 116.39723|   10|         0|       01|---GOV|         |      CHN|2018|        6.0|   7.0|
| 030|         CHN|20181012|

In [129]:
indexer = StringIndexer(inputCol="source", outputCol="src")
indexed = indexer.fit(indexed).transform(indexed)
indexed.show()

+----+------------+--------+---+--------------------+---------+--------+----------+-----+----------+---------+------+---------+---------+----+-----------+------+-----+
|code|country_code|   date8|day|             geoname|goldstein|latitude| longitude|month|quad_class|root_code|source|src_actor|tgt_actor|year|country_idx| gname|  src|
+----+------------+--------+---+--------------------+---------+--------+----------+-----+----------+---------+------+---------+---------+----+-----------+------+-----+
| 060|         UKR|20181012| 12|            Ukraine |      6.0|    49.0|      32.0|   10|         2|       06|   RUS|      RUS|         |2018|       44.0| 129.0| 21.0|
| 030|         RUS|20181012| 12| Russian Federation |      4.0|    60.0|     100.0|   10|         1|       03|RUSGOV|      RUS|         |2018|        9.0|  22.0| 34.0|
| 010|         CHN|20181012| 12| Beijing Beijing Shi|      0.0| 39.9075| 116.39723|   10|         0|       01|---GOV|         |      CHN|2018|        6.0|   7.0

In [130]:
indexer = StringIndexer(inputCol="src_actor", outputCol="src_act_idx")
indexed = indexer.fit(indexed).transform(indexed)
indexed.show()

+----+------------+--------+---+--------------------+---------+--------+----------+-----+----------+---------+------+---------+---------+----+-----------+------+-----+-----------+
|code|country_code|   date8|day|             geoname|goldstein|latitude| longitude|month|quad_class|root_code|source|src_actor|tgt_actor|year|country_idx| gname|  src|src_act_idx|
+----+------------+--------+---+--------------------+---------+--------+----------+-----+----------+---------+------+---------+---------+----+-----------+------+-----+-----------+
| 060|         UKR|20181012| 12|            Ukraine |      6.0|    49.0|      32.0|   10|         2|       06|   RUS|      RUS|         |2018|       44.0| 129.0| 21.0|        9.0|
| 030|         RUS|20181012| 12| Russian Federation |      4.0|    60.0|     100.0|   10|         1|       03|RUSGOV|      RUS|         |2018|        9.0|  22.0| 34.0|        9.0|
| 010|         CHN|20181012| 12| Beijing Beijing Shi|      0.0| 39.9075| 116.39723|   10|         0|

In [ ]:
# ,"geoname","source","src_actor"     ,"gname","src","src_act_idx"

In [140]:
indexed.select("src_act_idx").distinct().show()

+-----------+
|src_act_idx|
+-----------+
|      147.0|
|      160.0|
|        8.0|
|       67.0|
|       70.0|
|        0.0|
|       69.0|
|        7.0|
|      142.0|
|      154.0|
|      112.0|
|      124.0|
|      128.0|
|      108.0|
|      133.0|
|       88.0|
|       49.0|
|      155.0|
|      101.0|
|      116.0|
+-----------+
only showing top 20 rows



In [143]:
indexed = indexed.na.drop()
indexed.count()

19920

# One hot encoding

In [156]:
from pyspark.ml.feature import OneHotEncoderEstimator
encoder = OneHotEncoderEstimator(inputCols=["country_idx"],
                                 outputCols=["country_vec"])


In [157]:
from pyspark.sql import SQLContext
#from pyspark.mllib.linalg import Vectors
from pyspark.ml.linalg import Vectors, VectorUDT
model = encoder.fit(indexed)


In [158]:
encoded = model.transform(indexed)
encoded.show()

+----+------------+--------+---+--------------------+---------+--------+----------+-----+----------+---------+------+---------+---------+----+-----------+------+-----+-----------+----------------+
|code|country_code|   date8|day|             geoname|goldstein|latitude| longitude|month|quad_class|root_code|source|src_actor|tgt_actor|year|country_idx| gname|  src|src_act_idx|     country_vec|
+----+------------+--------+---+--------------------+---------+--------+----------+-----+----------+---------+------+---------+---------+----+-----------+------+-----+-----------+----------------+
| 060|         UKR|20181012| 12|            Ukraine |      6.0|    49.0|      32.0|   10|         2|       06|   RUS|      RUS|         |2018|       44.0| 129.0| 21.0|        9.0|(177,[44],[1.0])|
| 030|         RUS|20181012| 12| Russian Federation |      4.0|    60.0|     100.0|   10|         1|       03|RUSGOV|      RUS|         |2018|        9.0|  22.0| 34.0|        9.0| (177,[9],[1.0])|
| 010|         

In [159]:
from pyspark.ml.feature import OneHotEncoderEstimator
encoder = OneHotEncoderEstimator(inputCols=["gname"],
                                 outputCols=["gname_vec"])
from pyspark.sql import SQLContext
#from pyspark.mllib.linalg import Vectors
from pyspark.ml.linalg import Vectors, VectorUDT
model = encoder.fit(encoded)

encoded = model.transform(encoded)
encoded.show()

+----+------------+--------+---+--------------------+---------+--------+----------+-----+----------+---------+------+---------+---------+----+-----------+------+-----+-----------+----------------+-------------------+
|code|country_code|   date8|day|             geoname|goldstein|latitude| longitude|month|quad_class|root_code|source|src_actor|tgt_actor|year|country_idx| gname|  src|src_act_idx|     country_vec|          gname_vec|
+----+------------+--------+---+--------------------+---------+--------+----------+-----+----------+---------+------+---------+---------+----+-----------+------+-----+-----------+----------------+-------------------+
| 060|         UKR|20181012| 12|            Ukraine |      6.0|    49.0|      32.0|   10|         2|       06|   RUS|      RUS|         |2018|       44.0| 129.0| 21.0|        9.0|(177,[44],[1.0])| (1999,[129],[1.0])|
| 030|         RUS|20181012| 12| Russian Federation |      4.0|    60.0|     100.0|   10|         1|       03|RUSGOV|      RUS|     

In [160]:
from pyspark.ml.feature import OneHotEncoderEstimator
encoder = OneHotEncoderEstimator(inputCols=["src"],
                                 outputCols=["src_vec"])
from pyspark.sql import SQLContext
#from pyspark.mllib.linalg import Vectors
from pyspark.ml.linalg import Vectors, VectorUDT
model = encoder.fit(encoded)

encoded = model.transform(encoded)
encoded.show()

+----+------------+--------+---+--------------------+---------+--------+----------+-----+----------+---------+------+---------+---------+----+-----------+------+-----+-----------+----------------+-------------------+------------------+
|code|country_code|   date8|day|             geoname|goldstein|latitude| longitude|month|quad_class|root_code|source|src_actor|tgt_actor|year|country_idx| gname|  src|src_act_idx|     country_vec|          gname_vec|           src_vec|
+----+------------+--------+---+--------------------+---------+--------+----------+-----+----------+---------+------+---------+---------+----+-----------+------+-----+-----------+----------------+-------------------+------------------+
| 060|         UKR|20181012| 12|            Ukraine |      6.0|    49.0|      32.0|   10|         2|       06|   RUS|      RUS|         |2018|       44.0| 129.0| 21.0|        9.0|(177,[44],[1.0])| (1999,[129],[1.0])| (1386,[21],[1.0])|
| 030|         RUS|20181012| 12| Russian Federation |   

In [162]:
encoded = encoded.na.drop()
encoded.count()

19920

In [172]:
# from pyspark.ml.feature import OneHotEncoderEstimator
# encoder = OneHotEncoderEstimator(inputCols=["src_act_idx"],
#                                  outputCols=["src_act_vec"])
# from pyspark.sql import SQLContext
# #from pyspark.mllib.linalg import Vectors
# from pyspark.ml.linalg import Vectors, VectorUDT
# model = encoder.fit(encoded)

# encoded = model.transform(encoded)
# encoded.show()

In [174]:
indexed.select("src_act_idx").distinct().show()

Exception ignored in: <function JavaWrapper.__del__ at 0x1225c0440>
Traceback (most recent call last):
  File "/Users/meghnakurup/spark/python/pyspark/ml/wrapper.py", line 40, in __del__
    if SparkContext._active_spark_context and self._java_obj is not None:
AttributeError: 'OneHotEncoderEstimator' object has no attribute '_java_obj'


+-----------+
|src_act_idx|
+-----------+
|      147.0|
|      160.0|
|        8.0|
|       67.0|
|       70.0|
|        0.0|
|       69.0|
|        7.0|
|      142.0|
|      154.0|
|      112.0|
|      124.0|
|      128.0|
|      108.0|
|      133.0|
|       88.0|
|       49.0|
|      155.0|
|      101.0|
|      116.0|
+-----------+
only showing top 20 rows



In [175]:
encoded.show()

+----+------------+--------+---+--------------------+---------+--------+----------+-----+----------+---------+------+---------+---------+----+-----------+------+-----+-----------+----------------+-------------------+------------------+
|code|country_code|   date8|day|             geoname|goldstein|latitude| longitude|month|quad_class|root_code|source|src_actor|tgt_actor|year|country_idx| gname|  src|src_act_idx|     country_vec|          gname_vec|           src_vec|
+----+------------+--------+---+--------------------+---------+--------+----------+-----+----------+---------+------+---------+---------+----+-----------+------+-----+-----------+----------------+-------------------+------------------+
| 060|         UKR|20181012| 12|            Ukraine |      6.0|    49.0|      32.0|   10|         2|       06|   RUS|      RUS|         |2018|       44.0| 129.0| 21.0|        9.0|(177,[44],[1.0])| (1999,[129],[1.0])| (1386,[21],[1.0])|
| 030|         RUS|20181012| 12| Russian Federation |   

In [194]:
indexer = StringIndexer(inputCol="code", outputCol="code_idx")
encoded = indexer.fit(encoded).transform(encoded)
encoded.show()


+----+------------+--------+---+--------------------+---------+--------+----------+-----+----------+---------+------+---------+---------+----+-----------+------+-----+-----------+----------------+-------------------+------------------+--------+
|code|country_code|   date8|day|             geoname|goldstein|latitude| longitude|month|quad_class|root_code|source|src_actor|tgt_actor|year|country_idx| gname|  src|src_act_idx|     country_vec|          gname_vec|           src_vec|code_idx|
+----+------------+--------+---+--------------------+---------+--------+----------+-----+----------+---------+------+---------+---------+----+-----------+------+-----+-----------+----------------+-------------------+------------------+--------+
| 060|         UKR|20181012| 12|            Ukraine |      6.0|    49.0|      32.0|   10|         2|       06|   RUS|      RUS|         |2018|       44.0| 129.0| 21.0|        9.0|(177,[44],[1.0])| (1999,[129],[1.0])| (1386,[21],[1.0])|     4.0|
| 030|         RUS|2

In [195]:
# from pyspark.ml.feature import OneHotEncoderEstimator
encoder = OneHotEncoderEstimator(inputCols=["code_idx"],
                                 outputCols=["code_vec"])
from pyspark.sql import SQLContext
#from pyspark.mllib.linalg import Vectors
from pyspark.ml.linalg import Vectors, VectorUDT
model = encoder.fit(encoded)

encoded = model.transform(encoded)
encoded.show()

+----+------------+--------+---+--------------------+---------+--------+----------+-----+----------+---------+------+---------+---------+----+-----------+------+-----+-----------+----------------+-------------------+------------------+--------+----------------+
|code|country_code|   date8|day|             geoname|goldstein|latitude| longitude|month|quad_class|root_code|source|src_actor|tgt_actor|year|country_idx| gname|  src|src_act_idx|     country_vec|          gname_vec|           src_vec|code_idx|        code_vec|
+----+------------+--------+---+--------------------+---------+--------+----------+-----+----------+---------+------+---------+---------+----+-----------+------+-----+-----------+----------------+-------------------+------------------+--------+----------------+
| 060|         UKR|20181012| 12|            Ukraine |      6.0|    49.0|      32.0|   10|         2|       06|   RUS|      RUS|         |2018|       44.0| 129.0| 21.0|        9.0|(177,[44],[1.0])| (1999,[129],[1.0]

In [203]:
indexer = StringIndexer(inputCol="root_code", outputCol="root_idx")
encoded = indexer.fit(encoded).transform(encoded)
encoded.show()

+----+------------+--------+---+--------------------+---------+--------+----------+-----+----------+---------+------+---------+---------+----+-----------+------+-----+-----------+----------------+-------------------+------------------+--------+----------------+--------+
|code|country_code|   date8|day|             geoname|goldstein|latitude| longitude|month|quad_class|root_code|source|src_actor|tgt_actor|year|country_idx| gname|  src|src_act_idx|     country_vec|          gname_vec|           src_vec|code_idx|        code_vec|root_idx|
+----+------------+--------+---+--------------------+---------+--------+----------+-----+----------+---------+------+---------+---------+----+-----------+------+-----+-----------+----------------+-------------------+------------------+--------+----------------+--------+
| 060|         UKR|20181012| 12|            Ukraine |      6.0|    49.0|      32.0|   10|         2|       06|   RUS|      RUS|         |2018|       44.0| 129.0| 21.0|        9.0|(177,[44

In [204]:
# from pyspark.ml.feature import OneHotEncoderEstimator
encoder = OneHotEncoderEstimator(inputCols=["root_idx"],
                                 outputCols=["root_vec"])
from pyspark.sql import SQLContext
#from pyspark.mllib.linalg import Vectors
from pyspark.ml.linalg import Vectors, VectorUDT
model = encoder.fit(encoded)

encoded = model.transform(encoded)
encoded.show()

+----+------------+--------+---+--------------------+---------+--------+----------+-----+----------+---------+------+---------+---------+----+-----------+------+-----+-----------+----------------+-------------------+------------------+--------+----------------+--------+---------------+
|code|country_code|   date8|day|             geoname|goldstein|latitude| longitude|month|quad_class|root_code|source|src_actor|tgt_actor|year|country_idx| gname|  src|src_act_idx|     country_vec|          gname_vec|           src_vec|code_idx|        code_vec|root_idx|       root_vec|
+----+------------+--------+---+--------------------+---------+--------+----------+-----+----------+---------+------+---------+---------+----+-----------+------+-----+-----------+----------------+-------------------+------------------+--------+----------------+--------+---------------+
| 060|         UKR|20181012| 12|            Ukraine |      6.0|    49.0|      32.0|   10|         2|       06|   RUS|      RUS|         |20

# Assemble all the columns to be used as features and prepare data for training

In [301]:
from pyspark.ml.feature import VectorAssembler
va = VectorAssembler(inputCols=['country_vec','goldstein','latitude','longitude','quad_class'], outputCol='features')


In [302]:
dataset = va.transform(encoded)


In [303]:
dataset.select('features').show()

+--------------------+
|            features|
+--------------------+
|(181,[44,177,178,...|
|(181,[9,177,178,1...|
|(181,[6,178,179],...|
|(181,[6,177,178,1...|
|(181,[25,177,178,...|
|(181,[3,177,178,1...|
|(181,[3,177,178,1...|
|(181,[0,177,178,1...|
|(181,[7,177,178,1...|
|(181,[7,177,178,1...|
|(181,[5,177,178,1...|
|(181,[0,177,178,1...|
|(181,[7,177,178,1...|
|(181,[0,177,178,1...|
|(181,[1,177,178,1...|
|(181,[10,177,178,...|
|(181,[0,177,178,1...|
|(181,[1,177,178,1...|
|(181,[19,178,179]...|
|(181,[3,178,179],...|
+--------------------+
only showing top 20 rows



In [304]:
dataset = dataset.select('features','root_idx')

In [305]:
dataset.show()

+--------------------+--------+
|            features|root_idx|
+--------------------+--------+
|(181,[44,177,178,...|     6.0|
|(181,[9,177,178,1...|     2.0|
|(181,[6,178,179],...|     0.0|
|(181,[6,177,178,1...|     2.0|
|(181,[25,177,178,...|    17.0|
|(181,[3,177,178,1...|    11.0|
|(181,[3,177,178,1...|    11.0|
|(181,[0,177,178,1...|     3.0|
|(181,[7,177,178,1...|     7.0|
|(181,[7,177,178,1...|     2.0|
|(181,[5,177,178,1...|     3.0|
|(181,[0,177,178,1...|     9.0|
|(181,[7,177,178,1...|     5.0|
|(181,[0,177,178,1...|     3.0|
|(181,[1,177,178,1...|     8.0|
|(181,[10,177,178,...|     8.0|
|(181,[0,177,178,1...|     8.0|
|(181,[1,177,178,1...|     6.0|
|(181,[19,178,179]...|     0.0|
|(181,[3,178,179],...|     0.0|
+--------------------+--------+
only showing top 20 rows



# Split data set for training and testing

In [306]:
training, test = dataset.randomSplit([0.7, 0.3], seed = 1000)
# df4.randomSplit([1.0, 2.0], 24)

# Decision Tree

In [307]:
from pyspark.ml.classification import DecisionTreeClassifier
dt = DecisionTreeClassifier(labelCol = 'root_idx',featuresCol = 'features')
dt_model = dt.fit(training)
dt_predictions = dt_model.transform(test)

In [308]:
training.show()

+--------------------+--------+
|            features|root_idx|
+--------------------+--------+
|(181,[0,177,178,1...|     0.0|
|(181,[0,177,178,1...|     0.0|
|(181,[0,177,178,1...|     0.0|
|(181,[0,177,178,1...|     0.0|
|(181,[0,177,178,1...|     0.0|
|(181,[0,177,178,1...|     0.0|
|(181,[0,177,178,1...|     0.0|
|(181,[0,177,178,1...|     0.0|
|(181,[0,177,178,1...|     0.0|
|(181,[0,177,178,1...|     0.0|
|(181,[0,177,178,1...|     0.0|
|(181,[0,177,178,1...|     0.0|
|(181,[0,177,178,1...|     0.0|
|(181,[0,177,178,1...|     0.0|
|(181,[0,177,178,1...|     0.0|
|(181,[0,177,178,1...|     0.0|
|(181,[0,177,178,1...|     0.0|
|(181,[0,177,178,1...|     0.0|
|(181,[0,177,178,1...|     0.0|
|(181,[0,177,178,1...|     0.0|
+--------------------+--------+
only showing top 20 rows



In [309]:
dt_predictions.show()

+--------------------+--------+--------------------+--------------------+----------+
|            features|root_idx|       rawPrediction|         probability|prediction|
+--------------------+--------+--------------------+--------------------+----------+
|(181,[0,177,178,1...|     0.0|[53.0,0.0,0.0,0.0...|[1.0,0.0,0.0,0.0,...|       0.0|
|(181,[0,177,178,1...|     0.0|[53.0,0.0,0.0,0.0...|[1.0,0.0,0.0,0.0,...|       0.0|
|(181,[0,177,178,1...|     0.0|[53.0,0.0,0.0,0.0...|[1.0,0.0,0.0,0.0,...|       0.0|
|(181,[0,177,178,1...|     0.0|[53.0,0.0,0.0,0.0...|[1.0,0.0,0.0,0.0,...|       0.0|
|(181,[0,177,178,1...|     0.0|[53.0,0.0,0.0,0.0...|[1.0,0.0,0.0,0.0,...|       0.0|
|(181,[0,177,178,1...|     0.0|[53.0,0.0,0.0,0.0...|[1.0,0.0,0.0,0.0,...|       0.0|
|(181,[0,177,178,1...|     0.0|[53.0,0.0,0.0,0.0...|[1.0,0.0,0.0,0.0,...|       0.0|
|(181,[0,177,178,1...|     0.0|[53.0,0.0,0.0,0.0...|[1.0,0.0,0.0,0.0,...|       0.0|
|(181,[0,177,178,1...|     0.0|[53.0,0.0,0.0,0.0...|[1.0,0.0,0.0,

# Evaluating the model

In [310]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
dt_evaluator = MulticlassClassificationEvaluator(labelCol="root_idx",predictionCol = "prediction",metricName = "accuracy")

In [311]:
dt_accuracy = dt_evaluator.evaluate(dt_predictions)

In [312]:
dt_accuracy

0.8544912164401723

# Random Forest

In [313]:
from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier(featuresCol='features', labelCol='root_idx', numTrees=3)

In [314]:
rf_model = rf.fit(training)
# dt_model = dt.fit(training)
rf_predictions = rf_model.transform(test)
rf_evaluator = MulticlassClassificationEvaluator(labelCol="root_idx",predictionCol = "prediction",metricName = "accuracy")
rf_accuracy = rf_evaluator.evaluate(rf_predictions)

In [315]:
rf_accuracy

0.43138879681803116

In [316]:
# from pyspark.ml.classification import NaiveBayes

# SVM

In [350]:
from pyspark.ml.classification import LinearSVC
lsvc = LinearSVC(maxIter=5, regParam=0.1)

In [346]:
# lsvcModel = lsvc.fit(training)
training.show()

+--------------------+--------+
|            features|root_idx|
+--------------------+--------+
|(181,[0,177,178,1...|     0.0|
|(181,[0,177,178,1...|     0.0|
|(181,[0,177,178,1...|     0.0|
|(181,[0,177,178,1...|     0.0|
|(181,[0,177,178,1...|     0.0|
|(181,[0,177,178,1...|     0.0|
|(181,[0,177,178,1...|     0.0|
|(181,[0,177,178,1...|     0.0|
|(181,[0,177,178,1...|     0.0|
|(181,[0,177,178,1...|     0.0|
|(181,[0,177,178,1...|     0.0|
|(181,[0,177,178,1...|     0.0|
|(181,[0,177,178,1...|     0.0|
|(181,[0,177,178,1...|     0.0|
|(181,[0,177,178,1...|     0.0|
|(181,[0,177,178,1...|     0.0|
|(181,[0,177,178,1...|     0.0|
|(181,[0,177,178,1...|     0.0|
|(181,[0,177,178,1...|     0.0|
|(181,[0,177,178,1...|     0.0|
+--------------------+--------+
only showing top 20 rows



In [347]:
train = training.selectExpr("features as features", "root_idx as label")

In [348]:
train.show()

+--------------------+-----+
|            features|label|
+--------------------+-----+
|(181,[0,177,178,1...|  0.0|
|(181,[0,177,178,1...|  0.0|
|(181,[0,177,178,1...|  0.0|
|(181,[0,177,178,1...|  0.0|
|(181,[0,177,178,1...|  0.0|
|(181,[0,177,178,1...|  0.0|
|(181,[0,177,178,1...|  0.0|
|(181,[0,177,178,1...|  0.0|
|(181,[0,177,178,1...|  0.0|
|(181,[0,177,178,1...|  0.0|
|(181,[0,177,178,1...|  0.0|
|(181,[0,177,178,1...|  0.0|
|(181,[0,177,178,1...|  0.0|
|(181,[0,177,178,1...|  0.0|
|(181,[0,177,178,1...|  0.0|
|(181,[0,177,178,1...|  0.0|
|(181,[0,177,178,1...|  0.0|
|(181,[0,177,178,1...|  0.0|
|(181,[0,177,178,1...|  0.0|
|(181,[0,177,178,1...|  0.0|
+--------------------+-----+
only showing top 20 rows



In [351]:
# lsvcModel = lsvc.fit(train)
from pyspark.ml.classification import LogisticRegression, OneVsRest
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
ovr = OneVsRest(classifier=lsvc)

In [352]:
svmMod = ovr.fit(train)

In [353]:
svm_predictions = svmMod.transform(test)

In [354]:
svm_predictions.show()

+--------------------+--------+----------+
|            features|root_idx|prediction|
+--------------------+--------+----------+
|(181,[0,177,178,1...|     0.0|       0.0|
|(181,[0,177,178,1...|     0.0|       0.0|
|(181,[0,177,178,1...|     0.0|       0.0|
|(181,[0,177,178,1...|     0.0|       0.0|
|(181,[0,177,178,1...|     0.0|       0.0|
|(181,[0,177,178,1...|     0.0|       0.0|
|(181,[0,177,178,1...|     0.0|       0.0|
|(181,[0,177,178,1...|     0.0|       0.0|
|(181,[0,177,178,1...|     0.0|       0.0|
|(181,[0,177,178,1...|     0.0|       0.0|
|(181,[0,177,178,1...|     0.0|       0.0|
|(181,[0,177,178,1...|     0.0|       0.0|
|(181,[0,177,178,1...|     8.0|       0.0|
|(181,[0,177,178,1...|     0.0|       0.0|
|(181,[0,177,178,1...|     0.0|       0.0|
|(181,[0,177,178,1...|     0.0|       0.0|
|(181,[0,177,178,1...|     0.0|       0.0|
|(181,[0,177,178,1...|     8.0|       0.0|
|(181,[0,177,178,1...|     8.0|       0.0|
|(181,[0,177,178,1...|     8.0|       0.0|
+----------

In [355]:
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")

In [356]:
svm_predictions = svm_predictions.selectExpr("features as features", "root_idx as label","prediction as prediction")

In [358]:
accuracy = evaluator.evaluate(svm_predictions)
# lr_predictions.show()

In [359]:
accuracy

0.41083858137222407

# Logistic Regression

In [360]:
from pyspark.ml.classification import LogisticRegression, OneVsRest
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
lr = LogisticRegression(maxIter=10, tol=1E-6, fitIntercept=True)

In [361]:
ovr = OneVsRest(classifier=lr)

In [362]:
ovrModel = ovr.fit(train)

In [363]:
lr_predictions = ovrModel.transform(test)

In [364]:
lr_predictions.show()

+--------------------+--------+----------+
|            features|root_idx|prediction|
+--------------------+--------+----------+
|(181,[0,177,178,1...|     0.0|       0.0|
|(181,[0,177,178,1...|     0.0|       0.0|
|(181,[0,177,178,1...|     0.0|       0.0|
|(181,[0,177,178,1...|     0.0|       0.0|
|(181,[0,177,178,1...|     0.0|       0.0|
|(181,[0,177,178,1...|     0.0|       0.0|
|(181,[0,177,178,1...|     0.0|       0.0|
|(181,[0,177,178,1...|     0.0|       0.0|
|(181,[0,177,178,1...|     0.0|       0.0|
|(181,[0,177,178,1...|     0.0|       0.0|
|(181,[0,177,178,1...|     0.0|       0.0|
|(181,[0,177,178,1...|     0.0|       0.0|
|(181,[0,177,178,1...|     8.0|       0.0|
|(181,[0,177,178,1...|     0.0|       0.0|
|(181,[0,177,178,1...|     0.0|       0.0|
|(181,[0,177,178,1...|     0.0|       0.0|
|(181,[0,177,178,1...|     0.0|       0.0|
|(181,[0,177,178,1...|     8.0|       8.0|
|(181,[0,177,178,1...|     8.0|       8.0|
|(181,[0,177,178,1...|     8.0|       8.0|
+----------

In [365]:
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")

In [366]:
lr_predictions = lr_predictions.selectExpr("features as features", "root_idx as label","prediction as prediction")

In [367]:
accuracy = evaluator.evaluate(lr_predictions)
# lr_predictions.show()

In [368]:
accuracy

0.7543917799138217

In [340]:
!java -version

java version "1.8.0_211"
Java(TM) SE Runtime Environment (build 1.8.0_211-b12)
Java HotSpot(TM) 64-Bit Server VM (build 25.211-b12, mixed mode)


In [369]:
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
layers = [5,5, 4, 1]

In [370]:
trainer = MultilayerPerceptronClassifier(maxIter=100, layers=layers, blockSize=128, seed=1234)

In [374]:
# model = trainer.fit(train)

In [ ]:
result = model.transform(test)
predictionAndLabels = result.select("prediction", "label")
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
print("Test set accuracy = " + str(evaluator.evaluate(predictionAndLabels)))